<a href="https://colab.research.google.com/github/Vitish23/vitish/blob/main/LLM_Model_V_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 35.2 MB/s eta 0:00:00


In [ ]:
import cohere
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from annoy import AnnoyIndex
import numpy as np
import pandas as pd

In [ ]:
pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552449 sha256=ed20369f7b2d965dbee738034fecae5f58f13ad81c91a5e26d9870ceb6fc18e6
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [ ]:
question = "Tell me about South Indian Bank ?"
text = """The South Indian Bank Limited (SIB) is a major private sector bank headquartered at Thrissur in Kerala, India. As on March 31st 2023, the Bank had a network of 942 banking outlets (941 Branches & 1 Service Branch) and 1304 ATMs/CRMs (1170 ATMs and 134 CRMs) spanning in 26 States and 4 Union Territories).South Indian Bank was registered in 1929 under the Companies Act.[5] It commenced business on 29 January 1929 at Round South, Thrissur.

It was included in the second schedule of the Reserve Bank of India and became a scheduled Bank on 7 August 1946. SIB was the first scheduled Bank in the private sector in Kerala to get the license under Section 22 of the Banking Regulation Act, 1949 from the RBI on 17 June 1957.

The Bank has won the 'UiPath Automation Excellence Awards 2021' for Best Automation under crisis for business continuity."""

In [ ]:
## split into a list of paragraphs
texts = text.split('\n\n')

## clean up to remove empty spaces and new lines
texts = np.array([t.strip('\n') for t in texts if t])

co = cohere.Client('iddzxaFHN4geOTiM0Nl8Kk0Y4c6J637mUy9LaabU')

## get the embeddings
response = co.embed(
    texts=texts.tolist(),
).embeddings

In [ ]:
## chech the dimensions of the embeddings

embeds = np.array(response)

## create the search index, pass the size of embedding
search_index = AnnoyIndex(embeds.shape[1],'angular')
## Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10)  ## 10 trees
search_index.save('test.ann')

True

In [ ]:
def search_text(query):
  ## get the query's embeddings
  query_embed = co.embed(texts=[query]).embeddings

  ## retrieve the nearest neighbours
  similar_item_ids = search_index.get_nns_by_vector(query_embed[0],
                                                    10,
                                                    include_distances=True)

  search_results = texts[similar_item_ids[0]]

  return search_results

In [ ]:
results = search_text(question)

print(results[0])

The South Indian Bank Limited (SIB) is a major private sector bank headquartered at Thrissur in Kerala, India. As on March 31st 2023, the Bank had a network of 942 banking outlets (941 Branches & 1 Service Branch) and 1304 ATMs/CRMs (1170 ATMs and 134 CRMs) spanning in 26 States and 4 Union Territories).South Indian Bank was registered in 1929 under the Companies Act.[5] It commenced business on 29 January 1929 at Round South, Thrissur.


In [ ]:
def ask_llm(questio, num_generations=1):

  ## search the text archive
  results = search_text(question)

  ## Get the top resuts
  context = results[0]

  ## prepare the prompt

  prompt = f"""
  More information about Sydney at Australia.com:
  {context}
   Questions:{question}

   Extract the answer of the question from the provided.
   of the text doesn't contain the answer,
   reply thst the anser is not available."""

  Prediction = co.generate(
      prompt=prompt,
      max_tokens=70,
      model="command-nightly",
      temperature=0.5,
      num_generations=num_generations
   )

  return Prediction.generations

In [ ]:
results = ask_llm(question,)
print(results[0])

 South Indian Bank is a major private sector bank headquartered in Thrissur, Kerala, India. As of March 31st, 2023, the bank had a network of 942 banking outlets.


In [ ]:
question = "When was South Indian Started "
results = ask_llm(question,)
print(results[0])

 The answer is South Indian Bank was registered in 1929 under the Companies Act.


In [ ]:
question = "why should I join South Indian Bank ?"
results = ask_llm(question,)
print(results[0])

 The answer to why you should join South Indian Bank is not available in the provided text.
